In [52]:
import json;import datetime;from datetime import timedelta;import os, zipfile;
import pandas as pd;from scipy.optimize import curve_fit;from scipy.stats import chisquare
import numpy as np;import matplotlib.pyplot as plt
from LifeBuddyWebApp import create_app, db
from LifeBuddyWebApp.models import User, Post, Health_description, Health_measure
proxyApp=create_app()
ctx=proxyApp.app_context()
ctx.push()
from sqlalchemy import func

from bokeh.plotting import figure, output_file, show
import pandas as pd
from bokeh.resources import CDN
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, Text, Span
from bokeh.io import curdoc
from bokeh.themes import built_in_themes, Theme

from bokeh.layouts import row
import time
import pytz
import zoneinfo
from pytz import timezone

C:\tools\miniconda3\envs\lifebuddy\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [2]:
#filter on user data only
base_query_health_description=db.session.query(Health_description).filter(Health_description.user_id==1)
# if current_user.id==2:
df_health_description=pd.read_sql(str(base_query_health_description)[:-1]+str(1),db.session.bind)
# else:
#     df_health_description=pd.read_sql(str(base_query_health_description)[:-1]+str(current_user.id),db.session.bind)

In [3]:
#This stays exactly
colNames=[i[len('health_description_'):] for i in list(df_health_description.columns)]
col_names_dict={i:j for i,j in zip(list(df_health_description.columns),colNames)}
df_health_description.rename(columns=col_names_dict, inplace=True)

In [4]:
def plot_text_format(x):
    return ('%.1f' % x).rstrip('0').rstrip('.')

In [44]:
#This stays exactly
df1=df_health_description.loc[df_health_description.metric1_carido<100]
df1=df1.sort_values(by=['datetime_of_activity'])
obs_x1=[ datetime.datetime.strptime(date_string,'%Y-%m-%d %H:%M:%S.%f') for date_string in df1.datetime_of_activity]
obs_y1=df1.metric1_carido
obs_y1_formatted=[ plot_text_format(i) for i in obs_y1]

date_end=datetime.datetime.strptime(df1.datetime_of_activity.to_list()[-1],'%Y-%m-%d %H:%M:%S.%f')
date_end=date_end+ timedelta(days=5)
date_start=date_end- timedelta(days=7)

In [45]:
#get x and y's for activity
df2=df_health_description.loc[(df_health_description.var_type=='Activity') ]
#1 time (obs_x)
obs_x2=[ datetime.datetime.strptime(date_string,'%Y-%m-%d %H:%M:%S.%f') for date_string in df2.datetime_of_activity]
#3 activity (obs_y2)
obs_y2=df2.var_activity.to_list()

In [96]:
fig1=figure(toolbar_location=None,tools='xwheel_zoom,xpan',active_scroll='xwheel_zoom',
            x_range=(date_start,date_end),y_range=(-10,90),width=800, height=300)

#add cardio_metric1
circle=fig1.circle(obs_x1,obs_y1, legend_label="Cardio Performance", fill_color='#c77711', line_color=None,
                  size=20)
source1 = ColumnDataSource(dict(x=obs_x1, y=obs_y1, text=obs_y1_formatted))
glyph1 = Text(text="text",text_font_size={'value': '10px'},x_offset=-10, y_offset=5)
fig1.add_glyph(source1, glyph1)

for a,b in zip(obs_x2,obs_y2):
    #add activity data
    source2 = ColumnDataSource(dict(x=[a], y=[80], text=[b]))
    glyph2 = Text(text="text", text_color="#414444", text_font_size={'value': '10px'},
                 x_offset=-10,angle=-1.58)

    #add line for activity data
    line_start_time=time.mktime(a.timetuple())*1000
    important_time = Span(location=line_start_time, dimension='height', line_color="#414444", line_dash='dashed', line_width=1)
    fig1.add_glyph(source2, glyph2)
    fig1.add_layout(important_time)

fig1.add_glyph(source2, glyph2)
fig1.ygrid.grid_line_color = None
fig1.yaxis.major_label_text_color = None
fig1.yaxis.major_tick_line_color = None
fig1.yaxis.minor_tick_line_color = None

fig1.legend.background_fill_color = "#578582"
fig1.legend.background_fill_alpha = 0.2
fig1.legend.border_line_color = None

curdoc().theme = Theme(filename=r"D:\OneDrive\Documents\professional\20210710lifeBuddy\chart_theme_2.yml")
show(fig1)

ERROR:bokeh.core.validation.check:E-1020 (BAD_EXTRA_RANGE_NAME): An extra range name is configued with a name that does not correspond to any range: x_range_name='2021-08-18 14:04:09.641000' [Legend(id='25695', ...)]


Timeline Records table scratch work

In [97]:
#filter on user data only
# base_query_health_description=db.session.query(Health_description).filter(Health_description.user_id==1)
# if current_user.id==2:
# df_health_description=pd.read_sql(str(base_query_health_description)[:-1]+str(1),db.session.bind)
df_health_description.head()

,id,datetime_of_activity,var_activity,var_type,var_periodicity,var_timezone_utc_delta_in_mins,time_stamp_utc,user_id,source_filename,metric1_carido,metric2_session_duration,metric3,metric4,metric5,note
0,1,2021-06-09 12:08:34.635000,YOGA,heart rate,seconds,-240.0,2021-08-22 15:13:51.668660,1,training-session-2021-06-09-6138678344-024ee81...,21.733782,359.0,NaN,None,None,None
1,2,2021-07-19 09:58:41.657000,RUNNING,heart rate,seconds,120.0,2021-08-22 15:13:51.684289,1,training-session-2021-07-19-6283868904-0286c7f...,8.914898,888.0,NaN,None,None,None
2,3,2021-04-01 10:53:23.421000,TREADMILL_RUNNING,heart rate,seconds,-240.0,2021-08-22 15:13:51.715539,1,training-session-2021-04-01-5865912594-028fd1f...,15.078394,364.0,NaN,None,None,None
3,4,2021-03-21 13:01:19.783000,JOGGING,heart rate,seconds,-240.0,2021-08-22 15:13:51.746789,1,training-session-2021-03-21-5816145734-02a0a7b...,11.016714,1080.0,NaN,None,None,None
4,5,2021-03-29 12:26:53.276000,WALKING,heart rate,seconds,-240.0,2021-08-22 15:13:51.784548,1,training-session-2021-03-29-5853175779-034ad3a...,NaN,75.0,NaN,None,None,None


In [128]:
df_sub=df_health_description[['id', 'datetime_of_activity', 'var_activity','metric1_carido']].copy()
# df_health_description.datetime_of_activity.to_list()
# df_health_description.var_activity.to_list()
df_sub.values.tolist()[0][1]

'2021-06-09 12:08:34.635000'

In [129]:
df_sub.datetime_of_activity=df_sub['datetime_of_activity'].astype('datetime64[ns]')

In [130]:
df_sub.datetime_of_activity=pd.to_datetime(df_sub["datetime_of_activity"].dt.strftime('%m/%d/%Y %H:%M'))

In [125]:
df_sub.head()

,id,datetime_of_activity,var_activity
0,1,2021-06-09 12:08:00,YOGA
1,2,2021-07-19 09:58:00,RUNNING
2,3,2021-04-01 10:53:00,TREADMILL_RUNNING
3,4,2021-03-21 13:01:00,JOGGING
4,5,2021-03-29 12:26:00,WALKING


In [ ]:
db_sub.sort_values(by='datetime_of_activity')

In [160]:
table_lists=df_sub.values.tolist()

In [161]:
for i in table_lists:
    if i[3]=='nan':
        print('fire')
        i[3]='not exercise'
    else:
        i[3]='{0:.3g}'.format(i[3])

In [134]:
i=21.73378181755639
'{0:.3g}'.format(i)

'21.7'

In [149]:
nan_value=table_lists[4][3]
nan_value

'nan'

In [145]:
table_lists

[[1, Timestamp('2021-06-09 12:08:00'), 'YOGA', '21.7'],
 [2, Timestamp('2021-07-19 09:58:00'), 'RUNNING', '8.91'],
 [3, Timestamp('2021-04-01 10:53:00'), 'TREADMILL_RUNNING', '15.1'],
 [4, Timestamp('2021-03-21 13:01:00'), 'JOGGING', '11'],
 [5, Timestamp('2021-03-29 12:26:00'), 'WALKING', 'nan'],
 [6, Timestamp('2021-05-21 12:00:00'), 'STRENGTH_TRAINING', '22.2'],
 [7, Timestamp('2021-03-26 10:46:00'), 'WALKING', '28.6'],
 [8, Timestamp('2021-06-09 12:00:00'), 'WALKING', '0.103'],
 [9, Timestamp('2021-07-24 10:07:00'), 'RUNNING', '10.4'],
 [10, Timestamp('2021-04-21 09:59:00'), 'WALKING', '0.159'],
 [11, Timestamp('2021-06-07 11:58:00'), 'RUNNING', '5.95'],
 [12, Timestamp('2021-04-11 12:15:00'), 'JOGGING', '11.3'],
 [13, Timestamp('2021-03-26 11:03:00'), 'YOGA', '35.4'],
 [14, Timestamp('2021-05-15 11:09:00'), 'STRENGTH_TRAINING', '17'],
 [15, Timestamp('2021-06-07 12:08:00'), 'WALKING', '0.0361'],
 [16, Timestamp('2021-05-17 11:23:00'), 'RUNNING', '4.64'],
 [17, Timestamp('2021-08-0